In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# URL to scrape
url = "https://www.numbeo.com/cost-of-living/prices_by_city.jsp?displayCurrency=USD&itemId=24"

# Send a request to the URL and get the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find all the <thead> and <tbody> tags on the page
thead_tags = soup.find_all("thead")
tbody_tags = soup.find_all("tbody")

# Convert each <thead> tag into a list of column headings
thead_columns = []
for thead in thead_tags:
    header = [th.text.strip() for th in thead.find_all("th")]
    thead_columns.append(header)

# Convert each <tbody> tag into a pandas DataFrame with appropriate column headings
tbody_dataframes = []
for idx, tbody in enumerate(tbody_tags):
    rows = tbody.find_all("tr")
    data = []
    for row in rows:
        cols = row.find_all(["td", "th"])  # Some tables use <th> tags in the body as well
        row_data = [col.text.strip() for col in cols]
        data.append(row_data)
    df = pd.DataFrame(data, columns=thead_columns[idx])  # Use the corresponding <thead> columns as headings

    # Split the 'City' column into 'city' and 'country'
    df[['city', 'country']] = df['City'].str.split(',', 1, expand=True)
    df.drop(columns=['City'], inplace=True)  # Remove the original 'City' column

    # Split the 'country' column into 'state' and 'country'
    df[['state', 'country']] = df['country'].str.split(',', 1, expand=True)
    df['country'] = df.apply(lambda row: row['state'] if pd.isnull(row['country']) else row['country'], axis=1)
    df.drop(columns=['state'], inplace=True)  # Remove the 'state' column

    # Delete the 'Rank' column
    df.drop(columns=['Rank'], inplace=True)

    # Display the DataFrame
    # display(df)

    # Save the DataFrame to a CSV file
    csv_filename = f"./data/city_gas_prices.csv"
    df.to_csv(csv_filename, index=False)

#read, merge, rename columns, and perform minor cleaning
cost_df = pd.read_csv('./data/city_gas_prices.csv', header=0)
cities = pd.read_csv("./data/worldcities.csv", header=0)
cities_lat_long = pd.merge(cities, cost_df, on='city')
cities_lat_long = cities_lat_long.rename(columns={'Gasoline (1 liter)': 'gas_price_per_liter'})
cities_lat_long.drop_duplicates(ignore_index=True, subset=['city', 'gas_price_per_liter'], inplace=True)
cities_lat_long.dropna(subset=['gas_price_per_liter'], inplace=True)

#converting price per liter to gallon
def liter_to_gallon(value):
  gallon = value*3.785411784
  return gallon
cities_lat_long['gas_price_per_gallon'] = np.round(cities_lat_long['gas_price_per_liter'].map(liter_to_gallon), 2).map('{:0.2f}'.format)

# Create a world map to show global gas prices 
import folium
from folium.plugins import MarkerCluster
#empty map
world_map_cities= folium.Map(location=[0, 0], zoom_start=2, tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map_cities)
#for each coordinate, create circlemarker of user percent
for i in range(len(cities_lat_long)):
        lat = cities_lat_long.iloc[i]['lat']
        long = cities_lat_long.iloc[i]['lng']
        radius=6
        popup_text = """Country: {} <br>
                        City : {} <br>
                    Gas Price : ${} <br>"""
        popup_text = popup_text.format(cities_lat_long.iloc[i]['country_y'], 
                                        cities_lat_long.iloc[i]['city'],
                                   cities_lat_long.iloc[i]['gas_price_per_gallon']
                                   )
        folium.CircleMarker(location=[lat, long], radius=radius, popup=folium.Popup(popup_text, max_width=300), fill=True).add_to(marker_cluster)

#save the map as an html
world_map_cities.save("index.html")

#show the map
world_map_cities